# Setting things up

In [1]:
from general import *
from exp1 import *
import time
import pandas as pd
import multiprocessing as mp

random_state=42
np.random.seed(random_state)
cpu=mp.cpu_count() 
cpu

8

In [100]:
####
alpha=.05 #Significance level
m=700 #Training set size
n=1000 #Test set size

# Experiments

In [106]:
it=3
c=0.2
skew=0
gamma=0
theta=0
p=30
a=np.random.normal(0, 1, p).reshape(p,1)
#a/=np.sqrt((a**2).sum())
b=np.random.normal(0, 1, p).reshape(p,1)
#b/=np.sqrt((b**2).sum())

In [107]:
iterations=20

In [ ]:
eh=[]
erb=[]

for it in tqdm(range(iterations)):
    #Gen. training data
    Z_train=sample_z(m, p, random_state=2*it)
    X_train=sample_x(Z_train, b, random_state=2*it)
    Y_train=sample_y(X_train, Z_train, a, b, c, gamma, skew, random_state=2*it)

    #Gen. test data
    Z_test=sample_z(n, p, random_state=2*it+1)
    X_test=sample_x(Z_test, b, random_state=2*it+1)
    Y_test=sample_y(X_test, Z_test, a, b, c, gamma, skew, random_state=2*it+1)  

    #Fitting models
    g1 = g()
    g1.fit(X_train, Z_train, Y_train)
    g2 = g()
    g2.fit(None, Z_train, Y_train)
    g3 = g()
    g3.fit(None, Z_train, X_train)


    k=10
    h = GridSearchCV(KernelRidge(kernel='poly'), cv=2, n_jobs=-1, scoring='neg_mean_squared_error',
                     param_grid={"alpha": np.logspace(0,-k,k), "degree": [2]})

    h.fit(Z_train, g1.predict(X_train,Z_train))
    #print(h.best_params_)

    eh.append(np.mean((h.predict(Z_test)-Y_test)**2)-np.mean((g1.predict(X_test,Z_test)-Y_test)**2))
    erb.append(np.mean((g1.predict(mu_x(Z_test, b) + theta, Z_test)-Y_test)**2)-np.mean((g1.predict(X_test,Z_test)-Y_test)**2))

 15%|█▌        | 3/20 [00:04<00:19,  1.12s/it]

In [ ]:
plt.hist(eh, alpha=.5, density=True)
plt.hist(erb,alpha=.5, density=True)
plt.show()

In [ ]:
np.median(eh), np.median(erb)